# The MaxDet Correlation Matrix Completion Method

The MaxDet method (maximum determinant method) is a technique for completing a partially specified correlation matrix by selecting the values for the missing entries that maximize the determinant of the resulting matrix, subject to the matrix being positive semidefinite and having unit diagonal elements.

The chosen completion is the one that makes the determinant of the matrix as large as possible. The determinant of a correlation matrix is maximized when the variables are least correlated (i.e., most "independent") given the constraints.

The MaxDet resusult has some noteworthy properties:

* The MaxDet solution always exists and is unique if there is any valid completion at all.
* It is the maximum entropy solution, making it a natural choice when least bias or the most uncertainty is desired; this represents making the fewest possible assumptions beyond the constraints imposed by the known values.
* It coincides with the maximum likelihood estimate of the true correlation matrix, under a multivariate normal model, given the specified entries.
* The solution lies at the "analytic center" of the feasible region defined by the positive semidefiniteness constraints.

The MaxDet method is often cited in the literature and regulatory guidance as a suitable or recommended way to complete missing values, due to its attractive mathematical properties. For example, under Solvency II (EU), "Integration Technique 2" involves prudent bounds and positive definiteness for missing correlations, with central completions often suggested by MaxDet algorithms.

Let us illustrate the MaxDet method with some Python calcualtions. First, we load important Python libraries.

In [1]:
import numpy as np
import cvxpy as cp
import pandas as pd

STEP 1: Generate a random symmetric 5x5 matrix with ones on the diagonal.

In [2]:
# STEP 1: Generate random PSD correlation matrix
np.random.seed(42)
X = np.random.normal(size=(100, 5))
A = np.corrcoef(X, rowvar=False)  # guaranteed PSD

# Random matrix (not guaranteed to be PSD (will be fixed in a future version of this notebook):
print(pd.DataFrame(A))

          0         1         2         3         4
0  1.000000 -0.163433  0.044866 -0.101832 -0.126184
1 -0.163433  1.000000  0.122764  0.038215  0.057155
2  0.044866  0.122764  1.000000  0.008525  0.042819
3 -0.101832  0.038215  0.008525  1.000000  0.018358
4 -0.126184  0.057155  0.042819  0.018358  1.000000


Evaluate the generated correlations.

In [3]:
print("\nDeterminant of original A:", np.linalg.det(A))
print("Eigenvalues of A:", np.linalg.eigvalsh(A))
print("Diagonals of A:", np.diag(A))


Determinant of original A: 0.9257793320288289
Eigenvalues of A: [0.75068968 0.92287684 0.97741619 1.06361835 1.28539894]
Diagonals of A: [1. 1. 1. 1. 1.]


STEP 2: Create partial data  by masking 3 random off-diagonal values

In [4]:
mask = np.ones_like(A, dtype=bool)
np.fill_diagonal(mask, False)
missing_indices = np.random.choice(np.where(mask.flatten())[0], size=3, replace=False)
for idx in missing_indices:
    i, j = divmod(idx, A.shape[1])
    A[i, j] = np.nan
    A[j, i] = np.nan

# Matrix with missing entries:
print(pd.DataFrame(A))

          0         1         2         3         4
0  1.000000 -0.163433  0.044866 -0.101832 -0.126184
1 -0.163433  1.000000  0.122764  0.038215  0.057155
2  0.044866  0.122764  1.000000       NaN       NaN
3 -0.101832  0.038215       NaN  1.000000       NaN
4 -0.126184  0.057155       NaN       NaN  1.000000


STEP 3: MaxDet completion - maximize log(det(X)) subject to X PSD, X[i,j]=A[i,j] if specified, diag(X)=1

In [5]:
X = cp.Variable((5,5), symmetric=True)
constraints = [X >> 0]  # X is positive semidefinite
constraints += [X[i,i] == 1 for i in range(5)]
for i in range(5):
    for j in range(i+1, 5):
        if not np.isnan(A[i,j]):
            constraints += [X[i,j] == A[i,j], X[j,i] == A[j,i]]

problem = cp.Problem(cp.Maximize(cp.log_det(X)), constraints)
problem.solve()

# nMaxDet completed matrix:
print(pd.DataFrame(X.value))

          0         1         2         3         4
0  1.000000 -0.163433  0.044866 -0.101832 -0.126184
1 -0.163433  1.000000  0.122764  0.038215  0.057155
2  0.044866  0.122764  1.000000 -0.001686 -0.000778
3 -0.101832  0.038215 -0.001686  1.000000  0.013659
4 -0.126184  0.057155 -0.000778  0.013659  1.000000


Evaluate the end result.

In [6]:
print("\nDeterminant of completed matrix:", np.linalg.det(X.value))
print("Eigenvalues:", np.linalg.eigvalsh(X.value))
print("Diagonals:", np.diag(X.value))


Determinant of completed matrix: 0.9277259467272276
Eigenvalues: [0.76023523 0.89946869 0.98670955 1.07748289 1.27610364]
Diagonals: [1. 1. 1. 1. 1.]


The following references provide further details on this topic:
* L. Vandenberghe & S. Boyd, "Determinant Maximization with Linear Matrix Inequality Constraints" (1996) - Foundational work formalizing the maxdet problem in convex optimization, including efficient algorithms for matrix completion.
* D.I. Georgescu, "Explicit solutions to correlation matrix completion problems with block pattern" (2018) - Details theory of maxdet completions, including explicit solutions for certain block-patterned matrices.
* "Completing Correlation Matrices," The Actuary/Bank Underground (2018) - Practically oriented discussion of maxdet in risk modeling and capital requirement estimation; explains intuition and practical algorithms.